In [1]:
# relevant imports
import pandas as pd
import numpy as np

# Reading the SOEP Household data

Because the individual questionair file is quite big we will merge it soepwise with the weather and household data. Therefore we first read the hh files. All the required hh data can be found in two different data sets. These are merged in the following:

In [2]:
hbrutto_path = "./data/SOEP-CORE.v37eu_CSV/CSV/hbrutto.csv"
hl_path = "./data/SOEP-CORE.v37eu_CSV/CSV/hl.csv"

`hbrutto` contains meta data about the interviews. This includes data such as "day of interview", "location", etc. The `hl` file contains the interview responses of all waves in a long format.

In the following are the columns of interest with in the `hl` and `hbrutto` file with the corresponding meaning:

In [3]:
hl_var = [
    # keys for merging
    "syear", # survey year -> prim. key
    "hid", # hh id -> prim. key
    # hh control variables:
    "hlc0005_h", #[de] Monatliches HH-Netto-Einkommen [harmonisiert]
    "hlc0043", # Number Children
    "hlf0001_h", # Homewonership
    "hlk0056" # Type of interview
] # 

hbrutto_var = {
    "syear",  # survey year -> prim. key
    "hid",  # hh id -> prim. key
    "bula_h" # location -> bundesland
}

The two files are now read in to their respective dataframe.

In [4]:
df_hl = pd.read_csv(hl_path, usecols=hl_var)

In [5]:
df_hbrutto = pd.read_csv(hbrutto_path, usecols=hbrutto_var)

The two dataframes can now be merged on their primary key. According to the documentation of the db the prim. keys are `hid` and `syear`.

In [6]:
df_hh = df_hl.merge(df_hbrutto, how="inner", on=["hid", "syear"])

# Reading the SOEP Individual Data

Now we read the individual data. This dataset includes the target variable *SWB* as well as many other control variables. The datasets that contain the information used in this analysis are `ppathl` (tracking file) and `pl` (data)

In [7]:
ppathl_path = "./data/SOEP-CORE.v37eu_CSV/CSV/ppathl.csv"
pequiv_path = "./data/SOEP-CORE.v37eu_CSV/CSV/pequiv.csv"
pl_path = "./data/SOEP-CORE.v37eu_CSV/CSV/pl.csv"

In [8]:
ppathl_var = {
    "pid", # person id -> prim. key
    "syear", #survey year -> prim. key
    # relevant covariates
    "sex", # gender [1] female [2] male
    "gebjahr", # year of birth
    "partner" # [0] no partner, [1] spouse, [2] partner,
              # [3] Probably spouse, [4] probably partner
    # NOTE: join 1&3 and 2&4
}

pequiv_var = {
    "pid", # person id -> prim. key
    "syear", #survey year -> prim. key
    "d11109", #years of education: Year of eduction: numerical
    # NOTE: many nan values
    "m11124", # Disability status: [0] Not Disabled, [1] Disabled
    "e11103" # Labor Participation: [1] Full Time, [2] Part Time, [3] Not Working
}

pl_var = {
    # ids
    "pid",  # person id -> prim. key
    "syear",  #survey year -> prim. key
    "hid",  # hh id -> prim. key
    # target variable
    "plh0182", # Current life satisfaction [0-10]
    # relevant covariates
    "ptagin",  # day of interview
    "pmonin",  # month of interview
    "plh0171",  # Current Health: [1-10] (1=schlecht, 10=gut)
    "plb0021",  # Arbeitslos gemeldet: [2] No [1] Yes
    # NOTE: take max of plh0173 and plh0174
    "plh0175" # Satisfaction With Household Income
}

The `ppathl` contains the tracking data of a person. This includes for instance the age or marital status.

In [9]:
df_ppathl = pd.read_csv(ppathl_path, usecols=ppathl_var)

In [10]:
df_pequiv = pd.read_csv(pequiv_path, usecols=pequiv_var)

In the following we merge all the dataframes loaded into memory.

In [ ]:
soep = pd.read_csv(pl_path, usecols=pl_var)

## MERGE WITH OTHER DATASETS
# merge with tracking data
soep = soep.merge(df_ppathl, on=["syear", "pid"], how="inner")
# merge with pequiv (TODO what is this table)
soep = soep.merge(df_pequiv, on=["syear", "pid"], how="inner")
# merge with household
soep = soep.merge(df_hh, on=["syear", "hid"], how="inner")

## CALCULATE RELEVANT VARIABLES
# time stamp:
soep.rename({'syear':"year", 'pmonin':"month", 'ptagin':"day"}, axis=1, inplace=True)
soep["time"] = pd.to_datetime(soep[['year', 'month', 'day']], errors='coerce')
# drop unuseful columns:
soep.drop(['day', 'month', 'index'], axis=1, inplace=True)
# delete invalid time stamps as they cannot be merged with climate data:
soep = soep[soep['time'].notna()].reset_index()

## DROP DUPLICATE ROWS
soep.set_index(['pid', 'time'], inplace=True)
soep.sort_index(inplace=True)
soep = soep[~soep.index.duplicated(keep='first')]

## CALCULATE CHANGE IN SWB
soep['plh0182d'] = soep[['plh0182']].groupby('pid').diff(periods=1, axis=0)
soep.reset_index(inplace=True)

## PRINT EXAMPLE
soep.head()

# Data Cleanup

Most of the variables need some manual preparation when it comes to negative values. The conventions for missing values in SOEP is:

| Code | Label                                              | 
|------|----------------------------------------------------|
| -1   | No answer / don’t know                             |
| -2   | Does not apply                                     |
| -3   | Implausible value                                  |
| -4   | Inadmissable multiple response                     |
| -5   | Not included in this version of the questionnaire  |
| -6   | Version of questionnaire with modified filtering   |
| -7   | Only available in less restricted edition          |
| -8   | Question not part of the survey program this year  |



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

The number of children is a numerical value. If a person did not have children the value is $-2$. This is changed to 0.
All the other values are nan values. However, since the number of children does not change ofter, we can fill nan values with earlier observations.

In [ ]:
soep['hlc0043'] = soep['hlc0043'].replace({-2:0})
soep.loc[soep['hlc0043'] < 0, 'hlc0043'] = np.nan
soep['hlc0043'] = soep['hlc0043'].fillna(method='ffill')
soep[['hlc0043']].value_counts(dropna=False).plot(kind='bar')

The partner variable also has some redundant values. We have that $[3] \approx [1]$ and $[4] \approx [2]$. Furthermore, we have that $[5]$ stands for "unsure". 
Therefore, $[5]$ will be set to nan. 

In [ ]:
# clean values
soep.loc[soep['partner'] == 3, 'partner'] = 1
soep.loc[soep['partner'] == 4, 'partner'] = 2
# the rest is nan
soep.loc[(soep['partner'] == 5) | (soep['partner'] < 0), 'partner'] = np.nan
soep[['partner']].value_counts(dropna=False).plot(kind='bar')

With year of birth negative values are missing. We can use this variable to calculate the age of each person. Each age that is over 120 is concidered an outlier.

In [ ]:
soep.loc[soep['gebjahr'] < 1800, 'gebjahr'] = np.nan
soep["age"] = soep["year"] - soep["gebjahr"]
soep[soep["age"] > 120] = np.nan
soep['age'].hist(bins=20)

Years of education can also be padded with old values. 

In [ ]:
soep.loc[soep['d11109'] < 0, 'd11109'] = np.nan
soep['d11109'] = soep['d11109'].fillna(method='ffill', limit=2)
soep['d11109'].hist(bins=20)

In the follwoing variables the negative values add no information and are therefore just set to nan:

In [ ]:
cols = ['hlc0005_h', 'hlf0001_h', 'hlk0056', 'bula_h', 'hlk0056', 'hlf0001_h', 'sex']
for col in cols:
    soep.loc[soep[col] < 0, col] = np.nan

Save the dataframe

In [ ]:
soep.to_csv("./prod/soeplong.csv", index=True)